In [1]:
import findspark
findspark.init()

from pyspark.sql import functions as F

In [2]:
from pyspark.sql import SparkSession
from itertools import chain

import pandas as pd
pd.options.display.max_rows = 100

# Configure Spark session

In [7]:
def run():
    from pyspark import SparkContext, SparkConf

    conf = SparkConf()
    conf.setAppName('cars')
    conf.set("spark.driver.memory", "5g")
    conf.set("spark.driver.memoryOverhead", "0.10")
    sc = SparkContext(conf=conf)
    
    return sc

spark = SparkSession(run())

In [8]:
# turn on a dataframe view
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# Load data

In [9]:
df = spark.read.options(delimiter = '.').csv(path = "used_cars_data.csv", sep = ',',  header = True)

In [10]:
display(df)

vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
ZACNJABB5KPJ92081,35.1 in,null,null,null,SUV / Crossover,null,Bayamon,null,null,522,00960,[!@@Additional In...,I4,1300.0,I4,Solar Yellow,null,null,True,Jeep,41.2 in,12.7 gal,Gasoline,null,66.5 in,null,177.0,Black,null,null,null,True,null,18.3988,166.6 in,2019-04-06,YELLOW,237132766,-66.1582,https://static.ca...,['Quick Order Pac...,Jeep,5 seats,7.0,Renegade,null,"177 hp @ 5,750 RPM",23141.0,null,0,2.8,370599,Flagship Chrysler,null,"200 lb-ft @ 1,750...",A,9-Speed Automatic...,t83804,Latitude FWD,null,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
SALCJ2FX1LH858117,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,207,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-02-15,BLACK,265946296,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,8.0,Discovery Sport,null,"246 hp @ 5,500 RPM",46500.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
JF1VA2M67G9829723,35.4 in,null,null,null,Sedan,null,Guaynabo,17.0,null,1233,00969,null,H4,2500.0,H4,None,False,False,True,FIAT,43.3 in,15.9 gal,Gasoline,False,58.1 in,23.0,305.0,None,False,null,null,False,null,18.3467,180.9 in,2017-04-25,UNKNOWN,173473508,-66.1098,null,"['Alloy Wheels', ...",Subaru,5 seats,null,WRX STI,3.0,"305 hp @ 6,000 RPM",46995.0,False,0,null,370467,FIAT de San Juan,False,"290 lb-ft @ 4,000...",M,6-Speed Manual,t58994,Base,null,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
SALRR2RV0L2433391,37.6 in,null,null,null,SUV / Crossover,null,San Juan,null,null,196,00922,[!@@Additional In...,V6,3000.0,V6,Eiger Gray,null,null,True,Land Rover,39 in,23.5 gal,Gasoline,null,73 in,null,340.0,Gray (Ebony/Ebony...,null,null,null,True,null,18.4439,195.1 in,2020-02-26,GRAY,266911050,-66.0785,https://static.ca...,null,Land Rover,7 seats,11.0,Discovery,null,"340 hp @ 6,500 RPM",67430.0,null,0,3.0,389227,Land Rover San Juan,null,"332 lb-ft @ 3,500...",A,8-Speed Automatic...,t86074,V6 HSE AWD,null,AWD,All-Wheel Drive,115 in,87.4 in,2020
SALCJ2FXXLH862327,38.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,137,00922,[!@@Additional In...,I4,2000.0,I4,Narvik Black,null,null,True,Land Rover,39.1 in,17.7 gal,Gasoline,null,68 in,null,246.0,Black (Ebony),null,null,null,True,null,18.4439,181 in,2020-04-25,BLACK,270957414,-66.0785,https://static.ca...,['Adaptive Cruise...,Land Rover,7 seats,7.0,Discovery Sport,null,"246 hp @ 5,500 RPM",48880.0,null,0,3.0,389227,Land Rover San Juan,null,"269 lb-ft @ 1,400...",A,9-Speed Automatic...,t86759,S AWD,null,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
SALYK2EX1LA261711,37.1 in,null,null,null,SUV / Crossover,null,San Juan,null,null,242,00922,[!@@Additional In...,I4,2000.0,I4,Kaikoura Stone,False,False,True,Land Rover,40.2 in,16.6 gal,Gasoline,False,66.3 in,null,247.0,Brown (Ebony / Eb...,False,null,null,True,null,18.4439,188.9 in,2020-01-11,UNKNOWN,262940541,-66.0785,https://static.ca...,"['Leather Seats',...",Land Rover,5 seats,12.0,Range Rover Velar,null,"247 hp @ 5,500 RPM",66903.0,False,0,3.0,389227,Land Rover San Juan,False,"269 lb-ft @ 1

In [ ]:
# save to parquet
df.write.parquet("/tmp/parquet/cars.parquet")

del df 

In [66]:
# read from parquet
parDF=spark.read.parquet("/tmp/parquet/cars.parquet")

# Data overview

In [12]:
# shape
print(format(parDF.count(), ',d'), '\ncol cnt:', len(parDF.columns))

3,000,507 
col cnt: 66


In [13]:
# data types
print('Columns & datatypes:')
pd.DataFrame(parDF.dtypes, columns =['col_name', 'data_type'])

Columns & datatypes:


,col_name,data_type
0,vin,string
1,back_legroom,string
2,bed,string
3,bed_height,string
4,bed_length,string
5,body_type,string
6,cabin,string
7,city,string
8,city_fuel_economy,string
9,combine_fuel_economy,string


###### get count from descriptive stats

In [14]:
# descriptive 
descr = parDF.describe().toPandas()
descr

,summary,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,count,3000507,2840909,19679,429196,429191,2986589,63622,3000133,2508829,...,2918550,2912147,2880702,2884802,1158444,2853203,2848662,2828164,2830498,2946381
1,mean,Infinity,None,4081.25,5166.666666666667,63740.0,5000.0,36161.2,23104.127042857144,22.706573085279945,...,6938472.700991163,9321036.134121323,1.1656370838791057E7,1.1037576677955545E7,1.0958303484245954E7,1.267889072667737E7,1.3396662595330501E7,1.4679855323887112E7,1.3845013514002014E7,342090.8502279081
2,stddev,NaN,None,1653.3677751788923,1474.2229591663988,109272.00131781244,0.0,47620.57877850709,53347.21956367153,16.079865342342412,...,4.298426253523661E7,4.970049252064884E7,5.538545201250243E7,5.397217480572898E7,5.374599443231616E7,5.7648088615293436E7,5.918315832965848E7,6.178457258018881E7,6.009541186023207E7,9667363.89268572
3,min,Heated Windshield Washer nozzles,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,'Blind Spot Monitoring',WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,& belt. Recently changed the oil,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,...,,,,,,,,,,
4,max,type: SUV,V8 Flex Fuel Vehicle,Volkswagen,V8 Flex Fuel Vehicle,White,White,V8,private seller,V8,...,~BACK UP CAMERA~,~~ VERY LOW MILES ~~,~GM CERTIFIED,~~~ Call (502) 695-9050 to check availability ...,~~~ Call (502) 695-9050 to check availability ...,~PUSH BUTTON START~,~~~ Call (502) 695-9050 to check availability ...,~PUSH BUTTON START~,~GM CERTIFIED,~~~ Call (502) 695-9050 to check availability ...


In [15]:
# get count (descending order)
descr_trans = descr.T
descr_trans.columns = descr_trans.iloc[0]

descr_trans = descr_trans.drop(descr_trans.index[0])
descr_trans['count'] = descr_trans['count'].astype('int64') 
descr_trans['count'].sort_values(ascending=False)

vin                        3000507
city                       3000133
daysonmarket               3000126
dealer_zip                 3000126
exterior_color             2996558
body_type                  2986589
franchise_dealer           2985724
model_name                 2975734
interior_color             2973804
make_name                  2972421
price                      2969827
sp_name                    2967403
longitude                  2964886
sp_id                      2963782
savings_amount             2959047
listing_id                 2954554
year                       2946381
listing_color              2939284
is_new                     2935768
seller_rating              2931478
mileage                    2929166
engine_cylinders           2928909
engine_type                2926986
listed_date                2925745
description                2922226
fuel_type                  2921440
transmission               2918550
transmission_display       2912147
latitude            

In [16]:
parDF.where(F.col('city').isNotNull()).count()

3000133

###### get top n values by count from every column

In [17]:
descr_trans

summary,count,mean,stddev,min,max
vin,3000507,Infinity,NaN,Heated Windshield Washer nozzles,type: SUV
back_legroom,2840909,None,None,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
bed,19679,4081.25,1653.3677751788923,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,Volkswagen
bed_height,429196,5166.666666666667,1474.2229591663988,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8 Flex Fuel Vehicle
bed_length,429191,63740.0,109272.00131781244,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,White
body_type,2986589,5000.0,0.0,'Blind Spot Monitoring',White
cabin,63622,36161.2,47620.57877850709,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8
city,3000133,23104.127042857144,53347.21956367153,& belt. Recently changed the oil,private seller
city_fuel_economy,2508829,22.706573085279945,16.079865342342412,WE ARE AWAY'S LOOKING FOR CLEAN CARS AND TRU...,V8
combine_fuel_economy,86,6034.0,6862.30777508558,'Backup Camera',V6


In [18]:
n=10

def get_value_cnt(col, n=n, asc=True):

    cnt = dict(parDF.select(col).groupBy(col).count().orderBy('count', ascending=asc).take(n))
    return cnt

In [19]:
descr_trans['top_n_values'] = [get_value_cnt(col) for col in parDF.columns]

In [20]:
pd.set_option('display.max_colwidth', None)

In [21]:
descr_trans[['count', 'top_n_values']].sort_values(by='count', ascending=False)

summary,count,top_n_values
vin,3000507,"{'3GCUYGED8LG371406': 1, '1ZVHT82HX85193104': 1, '1FTFW1ET3CFA74788': 1, '1C4RJFLT7LC409726': 1, '1FTEW1E49LFC19305': 1, '1N4BL4EV2KC190225': 1, '1FTEW1EP9LKE15679': 1, '1FMCU0F63LUA55533': 1, '3FA6P0G70LR170135': 1, 'SHHFK7H66LU222102': 1}"
city,3000133,"{'Grove Hill': 1, 'Prairieville': 1, 'Halethorpe': 1, 'Terra Ceia': 1, 'Tumwater': 1, 'Grand Terrace': 1, 'West Henrietta': 1, 'Newmarket': 1, ' and any emission testing charge. Prices are subject to change. While every reasonable effort is made to ensure the accuracy of this information': 1, 'Wenatchee': 1}"
daysonmarket,3000126,"{'1897': 1, '12 gal': 1, '1512': 1, '1265': 1, '1361': 1, '1280': 1, '1572': 1, '1418': 1, '2034': 1, '1880': 1}"
dealer_zip,3000126,"{'10452': 1, '63038': 1, '97321': 1, ' and any emission testing charge. Prices are subject to change. While every reasonable effort is made to ensure the accuracy of this information': 1, ' Hill Hold Control and Electric Parking Brake|Tires: P255/35R20 Summer|Steel Spare Wheel|Compact Spare Tire Mounted Inside Under Cargo|Clearcoat Paint|Express Open/Close Sliding And Tilting Glass 1st Row Sunroof w/Sunshade|Body-Colored Front Bumper|Body-Colored Rear Bumper|Chrome Side Windows Trim and Black Front Windshield Trim|Aluminum Door Handles|Aluminum Power Heated Auto Dimming Side Mirrors w/Power Folding and Turn Signal Indicator|Fixed Rear Window w/Defroster|Light Tinted Glass|Speed Sensitive Rain Detecting Variable Intermittent Wipers w/Heated Jets|Front Windshield -inc: Sun Visor Strip|Galvanized Steel/Aluminum Panels|Lip Spoiler|Metal-Look Grille w/Chrome Surround|Front license plate bracket|Trunk Rear Cargo Access|Programmable Projector Beam Led Low Beam Daytime Running Auto-Leveling Headlamps w/Delay-Off|Front And Rear Fog Lamps|Perimeter/Approach Lights|LED Brakelights|Laminated Glass|Audio Theft Deterrent|Window Grid Diversity Antenna|Bose Surround Sound System w/Subwoofer -inc: AudioPilot': 1, '30088': 1, '08861': 1, '1.0': 1, '32119': 1, '06420': 1}"
exterior_color,2996558,"{'Phnatom Black': 1, '762 lbs.|Dash trim: Metal-look|Cupholders: Front and rear|Cylinders: 4|Daytime running lights: Daytime running lights|Max Torque RPM: 4': 1, 'Lunar Blue Meta': 1, 'CPO benefits: 172-point Inspection 12-month/12': 1, 'Tempest Blue Metallic Public wholesale': 1, 'Monza Red': 1, '000.00': 1, 'Army_green': 1, 'FORWARD COLLISION ALERT (Included and only available with (WPP) Driver Confidence II Package.) ': 1, ' Very Original and Never Restored': 1}"
body_type,2986589,"{' a cassette deck for all of the thrift store hounds. Additional features include ABS brakes': 1, ' 3.87 ratio|GVWR': 1, ' we are not responsible for any errors or omissions contained on these pages. Please verify any information in question with Platinum Auto Group[!@@Additional Info@@!]Engine: 3.5L V6 DOHC 24 Valve -inc: VVT-iW intake and VVT-i exhaust|Front-wheel drive|2.56 axle ratio|Battery w/Run Down Protection|Gas-Pressurized Shock Absorbers|Front And Rear Anti-Roll Bars|Electric Power-Assist Speed-Sensing Steering|16 Gal. Fuel Tank|Quasi-Dual Stainless Steel Exhaust w/Chrome Tailpipe Finisher|Strut Front Suspension w/Coil Springs|Double Wishbone Rear Suspension w/Coil Springs|4-Wheel Disc Brakes w/4-Wheel ABS': 1, ' 'Heated Seats'': 1, ' ABS (4-Wheel)': 1, ' and any emission testing charge. Prices are subject to change. While every reasonable effort is made to ensure the accuracy of this information': 1, 'Camry': 1, ' Front Full Feature Leather Appointed Bucket * Heated seats * Power driver and front passenger seat adjusters * 2-position driver memory * Floor console * Wireless charging * Navigation With ': 1, ' we are not responsible for any errors or omissions contained on these pages. Please verify any information in question with Platinum Auto Group[!@@Additional Info@@!]Rear axle': 1, 'EuroVan': 1}"
franchise_dealer,2985724,"{' Traction control. CARFAX One-Owner. INFINITI of Kansas City serves Kansas City': 1,

##### Columns below won't be investigated:

vin,
longitude,
savings_amount,
listing_id,
description,
latitude,
trimId,
main_picture_url,
frame_damaged,
theft_title,
salvage,
is_certified,
vehicle_damage_category,
bed_height,
bed_length,
cabin,
bed,
combine_fuel_economy,

( ͡° ͜ʖ ͡°)


# Data extraction

In [67]:
# create a list of extracted variables
ext_v = []

### boolean variables

if True in string = 1

if False in string = 0

something else = -1

In [68]:
# list of boolean variables
bool_v = ['franchise_dealer', 'is_new', 'fleet', 'has_accidents', 'isCab', 'is_cpo', 'is_oemcpo']

In [69]:
# extract 1, 0, -1
for col in bool_v:
    
    ext_name = 'is_{}'.format(col)
    parDF = parDF.withColumn(ext_name,\
         F.when(\
                F.lower(\
                F.col('{}'.format(col))).contains('true'), F.lit(1))\
                             .when(\
                                    F.lower(\
                                    F.col('{}'.format(col))).contains('false'), F.lit(0)).otherwise(F.lit(-1)))
    
    if ext_name not in ext_v:
        ext_v.append(ext_name)

In [70]:
# check

def check(prefix, variables):
    
    is_cols = [prefix + x for x in variables]
    source_vs_is = list(chain.from_iterable(zip(variables, is_cols)))

    return parDF.select(source_vs_is).limit(20).toPandas()

check('is_', bool_v)

,franchise_dealer,is_franchise_dealer,is_new,is_is_new,fleet,is_fleet,has_accidents,is_has_accidents,isCab,is_isCab,is_cpo,is_is_cpo,is_oemcpo,is_is_oemcpo
0,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
1,True,1,True,1,False,0,False,0,False,0,None,-1,None,-1
2,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
3,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
4,True,1,False,0,True,1,False,0,True,1,None,-1,None,-1
5,True,1,False,0,False,0,False,0,False,0,None,-1,None,-1
6,True,1,True,1,False,0,False,0,False,0,None,-1,None,-1
7,True,1,True,1,None,-1,None,-1,None,-1,None,-1,None,-1
8,False,0,False,0,True,1,False,0,True,1,None,-1,None,-1
9,True,1,False,0,False,0,False,0,False,0,None,-1,None,-1


### continous and integer variables

In [71]:
conti_var = ['daysonmarket'
             ,'price'
             ,'year'
             ,'mileage'
             ,'engine_displacement'
             ,'height'
             ,'fuel_tank_volume'
             ,'front_legroom'
             ,'horsepower'
             ,'back_legroom'
             ,'width'
             ,'wheelbase'
             ,'length'
             ,'city_fuel_economy'
             ,'seller_rating'
             ,'maximum_seating'
             ,'highway_fuel_economy'
            ]

In [72]:
# extract float / int from continous variables
for col in conti_var:
    
    ext_name = 'conti_{}'.format(col)
    parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '^\d+(?:.\d+)?' , 0))
    
    if ext_name not in ext_v:
        ext_v.append(ext_name)

In [73]:
check('conti_', conti_var)

,daysonmarket,conti_daysonmarket,price,conti_price,year,conti_year,mileage,conti_mileage,engine_displacement,conti_engine_displacement,...,length,conti_length,city_fuel_economy,conti_city_fuel_economy,seller_rating,conti_seller_rating,maximum_seating,conti_maximum_seating,highway_fuel_economy,conti_highway_fuel_economy
0,314,314,23006.0,23006.0,2020,2020,5.0,5.0,2000.0,2000.0,...,176.4 in,176.4,22.0,22.0,5.0,5.0,5 seats,5,25.0,25.0
1,57,57,25911.0,25911.0,2020,2020,8.0,8.0,2500.0,2500.0,...,191.7 in,191.7,23.0,23.0,4.333333333333333,4.333333333333333,5 seats,5,34.0,34.0
2,72,72,61305.0,61305.0,2020,2020,3.0,3.0,3600.0,3600.0,...,204.3 in,204.3,17.0,17.0,4.75,4.75,7 seats,7,25.0,25.0
3,75,75,27565.0,27565.0,2020,2020,1101.0,1101.0,1200.0,1200.0,...,171.4 in,171.4,26.0,26.0,4.75,4.75,5 seats,5,30.0,30.0
4,83,83,18995.0,18995.0,2019,2019,13994.0,13994.0,2000.0,2000.0,...,161.3 in,161.3,23.0,23.0,4.333333333333333,4.333333333333333,5 seats,5,29.0,29.0
5,33,33,58994.0,58994.0,2018,2018,46774.0,46774.0,6200.0,6200.0,...,203.9 in,203.9,14.0,14.0,3.8181818181818183,3.8181818181818183,8 seats,8,21.0,21.0
6,448,448,25942.0,25942.0,2019,2019,8.0,8.0,2000.0,2000.0,...,161.3 in,161.3,23.0,23.0,4.333333333333333,4.333333333333333,5 seats,5,29.0,29.0
7,13,13,26860.0,26860.0,2021,2021,5.0,5.0,None,None,...,176.4 in,176.4,None,None,5.0,5.0,5 seats,5,None,None
8,6,6,13942.0,13942.0,2014,2014,73145.0,73145.0,3500.0,3500.0,...,190.6 in,190.6,19.0,19.0,4.375,4.375,5 seats,5,26.0,26.0
9,39,39,21993.0,21993.0,2020,2020,10226.0,10226.0,2000.0,2000.0,...,176.4 in,176.4,22.0,22.0,3.8181818181818183,3.8181818181818183,5 seats,5,25.0,25.0


In [74]:
# engine cylinders

# get number of cylinders 
col = 'engine_cylinders'
ext_name = "cnt_{}".format(col)

parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '(?=^\d)\d+|(?<=^.)\d+' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)

check('cnt_', ['engine_cylinders'])

,engine_cylinders,cnt_engine_cylinders
0,I4,4
1,I4,4
2,V6,6
3,I3,3
4,I4,4
5,V8,8
6,I4,4
7,I4,4
8,V6,6
9,I4,4


In [75]:
# get engine configuration
col = 'engine_cylinders'
ext_name = "config_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '(?=^.\d$).|(?=^.\d\d$).' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)
    
check('config_', ['engine_cylinders'])

,engine_cylinders,config_engine_cylinders
0,I4,I
1,I4,I
2,V6,V
3,I3,I
4,I4,I
5,V8,V
6,I4,I
7,I4,I
8,V6,V
9,I4,I


In [76]:
# get owner count
col = 'owner_count'
ext_name="cnt_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '^\d(?:\d)?\.\d$' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)

check('cnt_', ['owner_count'])

,owner_count,cnt_owner_count
0,None,None
1,None,None
2,None,None
3,None,None
4,1.0,1.0
5,None,None
6,None,None
7,None,None
8,3.0,3.0
9,1.0,1.0


In [77]:
get_value_cnt('cnt_owner_count', n=20, asc=False)

{'': 1160581,
 None: 860062,
 '1.0': 593680,
 '2.0': 235027,
 '3.0': 91544,
 '4.0': 33293,
 '5.0': 13114,
 '6.0': 4964,
 '7.0': 2076,
 '8.0': 1044,
 '10.0': 748,
 '9.0': 516,
 '12.0': 295,
 '23.0': 292,
 '11.0': 247,
 '25.0': 213,
 '0.0': 195,
 '26.0': 184,
 '27.0': 149,
 '28.0': 139}

#### Power

In [78]:
# get power
col = 'power'
ext_name = "hp_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '^\d*' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)

# get power RPM
ext_name = "rpm_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '(?<=@\s)(\d+.\d*)' , 0))
parDF = parDF.withColumn(ext_name, F.regexp_replace(F.col(ext_name), "[,]", ""))

if ext_name not in ext_v:
    ext_v.append(ext_name)

In [79]:
check('hp_', ['power'])

,power,hp_power
0,"161 hp @ 6,200 RPM",161
1,"181 hp @ 6,000 RPM",181
2,"310 hp @ 6,800 RPM",310
3,"137 hp @ 5,000 RPM",137
4,"166 hp @ 6,000 RPM",166
5,"420 hp @ 5,600 RPM",420
6,"166 hp @ 6,000 RPM",166
7,None,None
8,"290 hp @ 6,400 RPM",290
9,"161 hp @ 6,200 RPM",161


In [80]:
check('rpm_', ['power'])

,power,rpm_power
0,"161 hp @ 6,200 RPM",6200
1,"181 hp @ 6,000 RPM",6000
2,"310 hp @ 6,800 RPM",6800
3,"137 hp @ 5,000 RPM",5000
4,"166 hp @ 6,000 RPM",6000
5,"420 hp @ 5,600 RPM",5600
6,"166 hp @ 6,000 RPM",6000
7,None,None
8,"290 hp @ 6,400 RPM",6400
9,"161 hp @ 6,200 RPM",6200


#### Torque

In [81]:
# get power
col = 'torque'
ext_name = "tq_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '^\d*' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)

# get power RPM
ext_name = "rpm_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '(?<=@\s)(\d+.\d*)' , 0))
parDF = parDF.withColumn(ext_name, F.regexp_replace(F.col(ext_name), "[,]", ""))

if ext_name not in ext_v:
    ext_v.append(ext_name)

In [82]:
check('tq_', ['torque'])

,torque,tq_torque
0,"150 lb-ft @ 4,700 RPM",150
1,"185 lb-ft @ 4,320 RPM",185
2,"266 lb-ft @ 2,800 RPM",266
3,"162 lb-ft @ 2,500 RPM",162
4,"149 lb-ft @ 4,450 RPM",149
5,"460 lb-ft @ 4,100 RPM",460
6,"149 lb-ft @ 4,450 RPM",149
7,None,None
8,"261 lb-ft @ 4,400 RPM",261
9,"150 lb-ft @ 4,700 RPM",150


In [83]:
check('rpm_', ['torque'])

,torque,rpm_torque
0,"150 lb-ft @ 4,700 RPM",4700
1,"185 lb-ft @ 4,320 RPM",4320
2,"266 lb-ft @ 2,800 RPM",2800
3,"162 lb-ft @ 2,500 RPM",2500
4,"149 lb-ft @ 4,450 RPM",4450
5,"460 lb-ft @ 4,100 RPM",4100
6,"149 lb-ft @ 4,450 RPM",4450
7,None,None
8,"261 lb-ft @ 4,400 RPM",4400
9,"150 lb-ft @ 4,700 RPM",4700


#### Listed Date

In [84]:
# get listed_date
col = 'listed_date'
ext_name = "ext_{}".format(col) 
parDF = parDF.withColumn(ext_name, F.to_date('{}'.format(col)))

if ext_name not in ext_v:
    ext_v.append(ext_name)

In [85]:
check('ext_', ['listed_date'])

,listed_date,ext_listed_date
0,2019-11-02,2019-11-02
1,2020-07-16,2020-07-16
2,2020-07-01,2020-07-01
3,2020-06-28,2020-06-28
4,2020-06-20,2020-06-20
5,2020-08-09,2020-08-09
6,2019-06-21,2019-06-21
7,2020-08-29,2020-08-29
8,2020-09-05,2020-09-05
9,2020-08-03,2020-08-03


In [86]:
# List of created columns
ext_v

['is_franchise_dealer',
 'is_is_new',
 'is_fleet',
 'is_has_accidents',
 'is_isCab',
 'is_is_cpo',
 'is_is_oemcpo',
 'conti_daysonmarket',
 'conti_price',
 'conti_year',
 'conti_mileage',
 'conti_engine_displacement',
 'conti_height',
 'conti_fuel_tank_volume',
 'conti_front_legroom',
 'conti_horsepower',
 'conti_back_legroom',
 'conti_width',
 'conti_wheelbase',
 'conti_length',
 'conti_city_fuel_economy',
 'conti_seller_rating',
 'conti_maximum_seating',
 'conti_highway_fuel_economy',
 'cnt_engine_cylinders',
 'config_engine_cylinders',
 'cnt_owner_count',
 'hp_power',
 'rpm_power',
 'tq_torque',
 'rpm_torque',
 'ext_listed_date']

#### Data types

In [87]:
# created data types
print('Created columns & datatypes:')
pd.DataFrame(parDF[ext_v].dtypes, columns =['col_name', 'data_type'])

Created columns & datatypes:


,col_name,data_type
0,is_franchise_dealer,int
1,is_is_new,int
2,is_fleet,int
3,is_has_accidents,int
4,is_isCab,int
5,is_is_cpo,int
6,is_is_oemcpo,int
7,conti_daysonmarket,string
8,conti_price,string
9,conti_year,string


# Size decrease - Convert column types

In [88]:
import pyspark.sql.types as T
import re

In [89]:
# replace empty value with '-1' on all DataFrame columns
parDF=parDF.select([F.when(F.col(c)=="", "-1").otherwise(F.col(c)).alias(c) for c in parDF.columns])

### integer size decrease

In [90]:
def int_size_decrease(columns, df=parDF):
    
    # integer types
    byteType = range(-128, 128)
    shortType = range(-32768, 32768)
    integerType = range(-2147483648, 2147483648)
    longType = range(-9223372036854775808, 9223372036854775808) 
    
    for col in columns:
        
        print('processing : ', col, '\n')
                
#         min_v = int(re.findall('^\d+', parDF.agg({'{}'.format(col): 'min'}).collect()[0][0])[0])
        max_v = int(re.findall('^(?:-)?\d+', parDF.agg({'{}'.format(col): 'max'}).collect()[0][0])[0])
        

        if max_v in byteType:
            df = df.withColumn(col, F.col(col).cast(T.ByteType()))
                                           
        elif max_v in shortType:
            df = df.withColumn(col, F.col(col).cast(T.ShortType()))
                                               
        elif max_v in integerType:
            df = df.withColumn(col, F.col(col).cast(T.IntegerType()))
                                           
        elif max_v in longType:
            df = df.withColumn(col, F.col(col).cast(T.LongType()))
            
            
    df=df.na.fill(-1)
                
    return df

#### integers to reduce:

In [91]:
to_integer_list = ['is_franchise_dealer'
                   ,'is_is_new'
                   ,'is_fleet'
                   ,'is_has_accidents'
                   ,'is_isCab'
                   ,'is_is_cpo'
                   ,'is_is_oemcpo'
                   ,'conti_daysonmarket'
                   ,'conti_year'
                   ,'conti_mileage'
                   ,'cnt_engine_cylinders'
                   ,'conti_maximum_seating'
                   ,'cnt_owner_count'
                   ,'hp_power'
                   ,'rpm_power'
                   ,'tq_torque'
                   ,'rpm_torque']


In [92]:
# created data types
parDF=int_size_decrease(to_integer_list, parDF)

print('Created columns & datatypes:')
pd.DataFrame(parDF[ext_v].dtypes, columns =['col_name', 'data_type'])

processing :  is_franchise_dealer 

processing :  is_is_new 

processing :  is_fleet 

processing :  is_has_accidents 

processing :  is_isCab 

processing :  is_is_cpo 

processing :  is_is_oemcpo 

processing :  conti_daysonmarket 

processing :  conti_year 

processing :  conti_mileage 

processing :  cnt_engine_cylinders 

processing :  conti_maximum_seating 

processing :  cnt_owner_count 

processing :  hp_power 

processing :  rpm_power 

processing :  tq_torque 

processing :  rpm_torque 

Created columns & datatypes:


,col_name,data_type
0,is_franchise_dealer,tinyint
1,is_is_new,tinyint
2,is_fleet,tinyint
3,is_has_accidents,tinyint
4,is_isCab,tinyint
5,is_is_cpo,tinyint
6,is_is_oemcpo,tinyint
7,conti_daysonmarket,smallint
8,conti_price,string
9,conti_year,smallint


In [93]:
# random '-1' check for conti_year:
get_value_cnt('conti_year', n=10, asc=False)

{-1: 1066677,
 2020: 731940,
 2017: 202979,
 2019: 164796,
 2018: 122101,
 2021: 112434,
 2016: 80300,
 2015: 64299,
 2014: 54411,
 2013: 50991}

### Float's extraction

In [94]:
to_float_list = ['conti_price'
                 ,'conti_seller_rating'
                 ,'conti_engine_displacement'
                 ,'conti_height'
                 ,'conti_fuel_tank_volume'
                 ,'conti_front_legroom'
                 ,'conti_horsepower'
                 ,'conti_back_legroom'
                 ,'conti_width'
                 ,'conti_wheelbase'
                 ,'conti_length'
                 ,'conti_highway_fuel_economy'
                 ,'conti_city_fuel_economy'
                ]

In [95]:
def get_float(columns, df=parDF):
    
    for c in columns:
        print('processing : ', c, '\n')
        df = df.withColumn(c, F.col(c).cast(T.DoubleType()))
        
        if c not in ext_v:
            ext_v.append(c) 

    df=df.na.fill(-1)
                
    return df

In [96]:
parDF = get_float(to_float_list)

print('Created columns & datatypes:')
pd.DataFrame(parDF[ext_v].dtypes, columns =['col_name', 'data_type'])

processing :  conti_price 

processing :  conti_seller_rating 

processing :  conti_engine_displacement 

processing :  conti_height 

processing :  conti_fuel_tank_volume 

processing :  conti_front_legroom 

processing :  conti_horsepower 

processing :  conti_back_legroom 

processing :  conti_width 

processing :  conti_wheelbase 

processing :  conti_length 

processing :  conti_highway_fuel_economy 

processing :  conti_city_fuel_economy 

Created columns & datatypes:


,col_name,data_type
0,is_franchise_dealer,tinyint
1,is_is_new,tinyint
2,is_fleet,tinyint
3,is_has_accidents,tinyint
4,is_isCab,tinyint
5,is_is_cpo,tinyint
6,is_is_oemcpo,tinyint
7,conti_daysonmarket,smallint
8,conti_price,double
9,conti_year,smallint


In [97]:
# descriptive 
descr = parDF[ext_v].describe().toPandas()
descr

,summary,is_franchise_dealer,is_is_new,is_fleet,is_has_accidents,is_isCab,is_is_cpo,is_is_oemcpo,conti_daysonmarket,conti_price,...,conti_maximum_seating,conti_highway_fuel_economy,cnt_engine_cylinders,config_engine_cylinders,cnt_owner_count,hp_power,rpm_power,tq_torque,rpm_torque,ext_listed_date
0,count,3000507,3000507,3000507,3000507,3000507,3000507,3000507,3000507,3000507,...,3000507,3000507,3000507,2928909,3000507,3000507,3000507,3000507,3000507,1810054
1,mean,0.05570625231002627,-0.10962214052491795,-0.5830561301806662,-0.5820629646923003,-0.5839639767545951,-0.9227210601408362,-0.9316052253835768,76.0584611200707,655502.5732681626,...,2.9585056792068807,1608.1512008365414,5.6586126944546375,-0.9997280554649886,-0.10919521267572446,688022.2179868269,2911.223095296895,197.50049841576774,1762.8509025308056,None
2,stddev,0.9091200247932215,0.8034048924892719,0.6291126089543881,0.6111561034111961,0.6127341330138582,0.373417689061164,0.34578152684734986,109.48225073250823,1.3268587625726802E7,...,4.697996657361952,664899.6987892828,103.33290711586744,0.030372540974229832,1.6823527210547158,1.3778221541499916E7,2938.75276397684,1169.1979702949905,1980.3726109532493,None
3,min,-1,-1,-1,-1,-1,-1,-1,-1,-1.0,...,-1,-1.0,-1,,-1,-1,-1,-1,-1,1001-01-01
4,max,1,1,1,1,1,1,1,17800,2.81922054E8,...,127,2.81820048E8,24095,W,99,281960038,9000,32766,8000,3932-01-01


# Categorical variables extraction

### Unify fields by uppercase and strip

In [98]:
cat_list = ['city'
,'exterior_color'
,'body_type'
,'model_name'
,'interior_color'
,'make_name'
,'sp_name'
,'listing_color'
,'engine_type'
,'fuel_type'
,'transmission'
,'transmission_display'
,'trim_name'
,'wheel_system'
,'wheel_system_display'
,'major_options'
,'franchise_make'
]

In [99]:
def get_upper(column, df):
    df = df.withColumn(column, F.upper(F.col(column)))
    
    return df

def get_trim(column, df):
    df = df.withColumn(column, F.trim(F.col(column)))
    
    return df

In [100]:
for c in cat_list:
    
    parDF = get_upper(c, parDF)
    parDF = get_trim(c, parDF)
    
    if c not in ext_v:
        ext_v.append(c)  
    

In [101]:
# preview
pd.DataFrame(parDF.select(cat_list).take(10)).head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NORMAL,BLACK NOIR PEARL,SUV / CROSSOVER,TUCSON,BLACK,HYUNDAI,O'BRIEN HYUNDAI,BLACK,I4,GASOLINE,A,6-SPEED AUTOMATIC,SE AWD,AWD,ALL-WHEEL DRIVE,"['ALLOY WHEELS', 'BLUETOOTH', 'BACKUP CAMERA', 'ANDROID AUTO', 'CARPLAY']",HYUNDAI
1,RANTOUL,BLACK,SEDAN,FUSION,NONE,FORD,SHIELDS AUTO CENTER,BLACK,I4,GASOLINE,A,6-SPEED AUTOMATIC,SE FWD,FWD,FRONT-WHEEL DRIVE,"['BLUETOOTH', 'BLIND SPOT MONITORING', 'PARKING SENSORS']",FORD
2,INDIANAPOLIS,WHITE FROST TRICOAT,SUV / CROSSOVER,ENCLAVE,WHITE (CHESTNUT W/EBONY ACCENTS W/PERFORATED LEATHER-APPO),BUICK,RAY SKILLMAN NORTHEAST BUICK GMC,WHITE,V6,GASOLINE,A,9-SPEED AUTOMATIC,AVENIR AWD,4WD,FOUR-WHEEL DRIVE,"['LEATHER SEATS', 'SUNROOF/MOONROOF', 'NAVIGATION SYSTEM', 'TECHNOLOGY PACKAGE', 'BLUETOOTH', 'REMOTE START', 'CARGO PACKAGE', 'HEATED SEATS', 'MULTI ZONE CLIMATE CONTROL']",BUICK
3,INDIANAPOLIS,EBONY TWILIGHT METALLIC,SUV / CROSSOVER,ENCORE GX,BLACK (EBONY W/EBONY ACCENTS W/CLOTH W/LEATHERETTE SEAT T),BUICK,RAY SKILLMAN NORTHEAST BUICK GMC,UNKNOWN,I3,GASOLINE,CVT,CONTINUOUSLY VARIABLE TRANSMISSION,PREFERRED FWD,FWD,FRONT-WHEEL DRIVE,"['BLUETOOTH', 'COMFORT PACKAGE']",BUICK
4,RANTOUL,RED,SUV / CROSSOVER,ECOSPORT,NONE,FORD,SHIELDS AUTO CENTER,RED,I4,GASOLINE,A,6-SPEED AUTOMATIC,TITANIUM AWD,AWD,ALL-WHEEL DRIVE,"['SUNROOF/MOONROOF', 'NAVIGATION SYSTEM', 'BLUETOOTH', 'BACKUP CAMERA', 'BLIND SPOT MONITORING', 'PARKING SENSORS', 'HEATED SEATS']",FORD


### Convert dealer zip into integer

In [102]:
# preview
pd.DataFrame(parDF.select('dealer_zip').dtypes, columns =['col_name', 'data_type'])

,col_name,data_type
0,dealer_zip,string


#### convert into integer

In [103]:
# get dealer_zip
col = 'dealer_zip'
ext_name = "ext_{}".format(col)
parDF = parDF.withColumn(ext_name, F.regexp_extract('{}'.format(col), '\d+' , 0))

if ext_name not in ext_v:
    ext_v.append(ext_name)

In [104]:
# preview
parDF.select(['dealer_zip', 'ext_dealer_zip'])

dealer_zip,ext_dealer_zip
61761,61761
61866,61866
46219,46219
46219,46219
61866,61866
83713,83713
61866,61866
61761,61761
62208,62208
83713,83713


In [105]:
# convert into integer
parDF = int_size_decrease(['ext_dealer_zip'], parDF)

if 'ext_dealer_zip' not in ext_v:
    ext_v.append('ext_dealer_zip')

pd.DataFrame(parDF.select('ext_dealer_zip').dtypes, columns =['col_name', 'data_type'])

processing :  ext_dealer_zip 



,col_name,data_type
0,ext_dealer_zip,int


### major_options extraction

In [106]:
# preview
parDF.select('major_options').limit(n)

major_options
"['ALLOY WHEELS', ..."
"['BLUETOOTH', 'BL..."
"['LEATHER SEATS',..."
"['BLUETOOTH', 'CO..."
['SUNROOF/MOONROO...
"['LEATHER SEATS',..."
['SUNROOF/MOONROOF']
"['ALLOY WHEELS', ..."
['SUNROOF/MOONROO...
"['ALLOY WHEELS', ..."


In [107]:
# ...

In [108]:
if 'major_options' not in ext_v:
    ext_v.append('major_options')

# Final features list

In [109]:
pd.DataFrame(parDF[ext_v].dtypes, columns =['col_name', 'data_type'])

,col_name,data_type
0,is_franchise_dealer,tinyint
1,is_is_new,tinyint
2,is_fleet,tinyint
3,is_has_accidents,tinyint
4,is_isCab,tinyint
5,is_is_cpo,tinyint
6,is_is_oemcpo,tinyint
7,conti_daysonmarket,smallint
8,conti_price,double
9,conti_year,smallint


In [ ]:
# pivotDF = parDF.groupBy('body_type').pivot('ext_dealer_zip').avg('conti_price')

In [ ]:
# pivotDF.show(20)

In [110]:
spark.stop()